# Data Querying and Analysis

## Table of Contents

1. Introduction
2. Examples
3. References and Further Reading


<a id='2'></a>

## 1. Introduction

Generative AI can be used in various ways for Natural Language Querying, specifically for converting English to SQL:

1. **Direct Translation**: Converting natural language questions into SQL queries.
2. **Query Optimization**: Improving existing SQL queries based on natural language descriptions.
3. **Schema Understanding**: Interpreting database schemas from natural language descriptions.
4. **Error Correction**: Identifying and fixing errors in SQL queries based on natural language feedback.
5. 1. Query Translation: Converting natural language questions into formal query languages like SQL.
6. Query Generation: Creating complex queries based on high-level descriptions of data needs.
7. Result Interpretation: Translating query results into human-readable explanations.
8. Schema Understanding: Analyzing database schemas to inform query generation.
9. Error Handling: Identifying and explaining errors in queries or results.

Using Gen AI for this task offers several benefits:

- Increased accessibility for non-technical users
- Faster query development
- Reduced errors in complex queries
- Improved data exploration capabilities

**Key Terminology**:

- **Natural Language Processing (NLP)**: The branch of AI that deals with the interaction between computers and humans using natural language.
- **SQL (Structured Query Language)**: A standard language for managing and manipulating relational databases.
- **Schema**: The structure of a database, defining tables, fields, relationships, and constraints.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from rich.console import Console
import json
console = Console()

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
console.print(f"[dark_orange]Using OpenAI API key: {OPENAI_API_KEY[:12]}[/]")
# Set up OpenAI API key
client = OpenAI(api_key=OPENAI_API_KEY)



Using OpenAI API key: sk-proj-BXA7

In [2]:
def get_first_value(dict_variable):
    """The function clean takes a dictionary (dict_variable) as input and returns the value of the first key-value pair in the dictionary.

    Here's a breakdown:

    iter(dict_variable.values()) creates an iterator over the dictionary's values.
    next(...) retrieves the first value from the iterator.
    So, if you have a dictionary like {'a': 1, 'b': 2, 'c': 3}, calling clean on it would return 1, which is the value associated with the first key 'a'.

    Note that this function assumes the dictionary is not empty. If the dictionary is empty, next will raise a StopIteration exception.
    """
    return next(iter(dict_variable.values()))

<a id='3'></a>

## 2. Example 1: Simple Query Conversion


In [3]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a SQL expert. Convert English queries to SQL.",
        },
        {
            "role": "user",
            "content": "Convert this English query to SQL and output in JSON form: Find top 5 customers by revenue.",
        },
    ],
    response_format={"type": "json_object"},
)

print(response.choices[0].message.content)


{"query": "SELECT customer_id, SUM(revenue) AS total_revenue FROM customers GROUP BY customer_id ORDER BY total_revenue DESC LIMIT 5;"}


In [4]:
schema = """

sales
order_id
customer_id
sales

customer
customer_code
customer_name

"""


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a SQL expert. Convert English queries to SQL.",
        },
        {
            "role": "user",
            "content": "Convert this English query to SQL and output in JSON form: Find top 5 customers (customer names) by sales. The schema of this database is: {}".format(
                schema
            ),
        },
    ],
    response_format={"type": "json_object"},
)

print(response.choices[0].message.content)


{
  "sql": "SELECT c.customer_name FROM customer c JOIN sales s ON c.customer_code = s.customer_id GROUP BY c.customer_name ORDER BY SUM(s.sales) DESC LIMIT 5;"
}


This example demonstrates how Gen AI can convert a simple English query into SQL. The JSON output makes it easy for data engineers to parse and integrate the result into their workflows.


<a id='5'></a>

## 2. Example 2: Query Optimization and Cleaning


In [5]:
bad_query = """
SELECT * FROM orders WHERE order_date > '2023-01-01' AND customer_id IN (SELECT customer_id FROM customers WHERE country = 'USA')
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a SQL expert. Optimize SQL queries based on natural language descriptions.",
        },
        {
            "role": "user",
            "content": "Optimize this SQL query and output in JSON form: {} Optimization goal: Improve performance for a large dataset.".format(
                bad_query
            ),
        },
    ],
    response_format={"type": "json_object"},
)

print(json.loads(response.choices[0].message.content))

{'optimized_query': "SELECT o.* FROM orders o JOIN customers c ON o.customer_id = c.customer_id WHERE o.order_date > '2023-01-01' AND c.country = 'USA'"}


In [7]:
bad_query = """
SELECT *
FROM (
    SELECT c3.id, c3.date, c3.amt, c3.prod_name, c3.region, e.emp_name
    FROM region c3
    JOIN employees e ON c3.id = e.id
    WHERE e.active = 1
) AS subquery
JOIN (
    SELECT AVG(s.amt) AS avg_amt
    FROM sales s
) AS subquery2 ON subquery.amt > subquery2.avg_amt;
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a SQL expert. Given a SQL query, make it more readable. You may split the query into multiple steps with temp tables, and add naming conventions / column names. \
            Avoid complex joins and nested CTEs.",
        },
        {
            "role": "user",
            "content": "make it more readable. You may split the query into multiple steps with temp tables, and add naming conventions / column names. \
            Avoid complex joins and nested CTEs. Output in JSON form: {}".format(
                bad_query
            ),
        },
    ],
    response_format={"type": "json_object"},
)

good_query = get_first_value(json.loads(response.choices[0].message.content))

In [ ]:
good_query

In [ ]:
for g in good_query:
    print(g)
    print("----")

## 2. Example 3: Query Result Interpretation


In [8]:
# Simulated query result
query_result = """
| product_category | total_sales |
|-------------------|-------------|
| Electronics      | 1500000     |
| Clothing         | 980000      |
| Home & Garden    | 750000      |
| Books            | 320000      |
| Toys             | 280000      |
"""

prompt = f"Interpret the following query result and provide insights. Here's the result:\n{query_result}"

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

The query result provides a summary of total sales across different product categories. Here’s an interpretation of the results along with insights:

### Overview of Sales by Category

1. **Electronics**: 
   - **Total Sales**: $1,500,000
   - **Insight**: This category significantly outperforms all others, indicating a strong consumer demand for electronic products. This could be attributed to advancements in technology, seasonal promotions, or a trend towards smart products.

2. **Clothing**: 
   - **Total Sales**: $980,000
   - **Insight**: While clothing sales are robust, they are only about two-thirds of electronic sales. This suggests that while clothing is a staple category, it may not be experiencing the same growth or consumer interest as electronics. There may be opportunities to enhance marketing strategies or diversify offerings in this category.

3. **Home & Garden**: 
   - **Total Sales**: $750,000
   - **Insight**: This category has solid sales, possibly reflecting consu

<a id='4'></a>

## 2. Example 4: Generating Insights from Query Results


In [9]:
prompt = """Based on the following query result, suggest 3 relevant follow-up questions for further analysis:
{
  "category": ["Electronics", "Clothing", "Books"],
  "total_sales": [500000, 300000, 100000]
}"""

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Based on the provided query result, here are three relevant follow-up questions for further analysis:

1. **What are the sales trends over the past few months for each category?** 
   - This question aims to analyze how sales have changed over time within each category, which can help identify seasonal patterns or the impact of marketing campaigns.

2. **What are the demographics of the customers purchasing products in each category?** 
   - Understanding customer demographics can provide insights into target audiences for marketing strategies and product development.

3. **How do the total sales compare to previous periods or against competitors in each category?**
   - This question seeks to evaluate performance relative to historical data and market position, helping to assess the growth or decline in each category’s sales.


This example demonstrates how Gen AI can assist in query optimization, a crucial skill for data engineers working with large datasets and complex database structures.


<a id='7'></a>

## 3. References and Further Reading

1. OpenAI API Documentation: https://platform.openai.com/docs/
2. "Natural Language to SQL: A Survey" by Xu et al. (2022): https://arxiv.org/abs/2201.00307
3. "Improving Text-to-SQL Evaluation Methodology" by Zhong et al. (2020): https://arxiv.org/abs/1806.09029
4. SQL Performance Tuning by Peter Gulutzan and Trudy Pelzer
5. "Fundamentals of Database Systems" by Ramez Elmasri and Shamkant Navathe
